In [ ]:
from pyomo.environ import *
# import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

In [ ]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 48  # Number of execution scenarios                     (chi)

daily_demand = 100
demand_penalty = 15

M = 1e3  # Big M 

In [ ]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=125, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    # com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    # prod_max = {0:30, 1: 60, 2: 90, 3: 120, 4: 150}
    # prod_min = {0:0, 1: 30, 2: 60, 3: 90, 4: 120}
    # rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    
    com1_procure = Process(name='procure com1', prod_max=com1_process_capacity, conversion={com1_pur: -1, com1_in: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Procure com1', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Sell com1')
    com1_opt_procure = Process(name='procure optional com1', prod_max=75, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.01, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 1')
    # com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 2')
    
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Process the commodity through the location')
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    # 
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.1, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    
    com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=0.1, vopex=0.01, storage_capex=10, store_min=0.01, store_max= 40, prod_min=0.01, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=0.1, vopex=0.02, storage_capex=20, store_min=0.01, store_max= 80, prod_min=0.01, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=0.1, vopex=0.05, storage_capex=50, store_min=0.01, store_max= 200, prod_min=0.01, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 1')
    # com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 2')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_process, com1_store10, com1_store20, com1_store50, com1_loc1_send, com1_sell, com1_opt_procure}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_procure: cap_factor[['com1_procure']]})
    
   
    locset = [loc1]


    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 15
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc1 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc1 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_baseline', network= loc1, scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Scenario with perfect information')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    
    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    
    problem_mincost.first_stage_cost  = capex_process
    
    return scenario, problem_mincost

In [ ]:
def fix_variables(model):
    model.X_P['loc1','com1_loc1_send',0].fixed = True
    model.X_P['loc1','com1_loc1_send',0] = 0
    
    model.X_P['loc1','com1_process',0].fixed = True
    model.X_P['loc1','com1_process',0] = 1
    
    model.X_P['loc1','com1_store10',0].fixed = True
    model.X_P['loc1','com1_store10',0] = 1
    
    model.X_P['loc1','com1_store10_discharge',0].fixed = True
    model.X_P['loc1','com1_store10_discharge',0] = 1
    
    model.X_P['loc1','com1_store20',0].fixed = True
    model.X_P['loc1','com1_store20',0] = 1
    
    model.X_P['loc1','com1_store20_discharge',0].fixed = True
    model.X_P['loc1','com1_store20_discharge',0] = 1
    
    model.X_P['loc1','com1_store50',0].fixed = True
    model.X_P['loc1','com1_store50',0] = 0
    
    model.X_P['loc1','com1_store50_discharge',0].fixed = True
    model.X_P['loc1','com1_store50_discharge',0] = 0
    
    model.X_P['loc1','procure com1',0].fixed = True
    model.X_P['loc1','procure com1',0] = 1
    
    model.X_P['loc1','procure optional com1',0].fixed = True
    model.X_P['loc1','procure optional com1',0] = 1
    
    model.X_P['loc1','sell com1',0].fixed = True
    model.X_P['loc1','sell com1',0] = 1
    
    model.Cap_P['loc1','com1_loc1_send',0].fixed = True
    model.Cap_P['loc1','com1_loc1_send',0] = 0
    
    model.Cap_P['loc1','com1_process',0].fixed = True
    model.Cap_P['loc1','com1_process',0] = 100
    
    model.Cap_P['loc1','com1_store10',0].fixed = True
    model.Cap_P['loc1','com1_store10',0] = 8.0
    
    model.Cap_P['loc1','com1_store10_discharge',0].fixed = True
    model.Cap_P['loc1','com1_store10_discharge',0] = 1.25
    
    model.Cap_P['loc1','com1_store20',0].fixed = True
    model.Cap_P['loc1','com1_store20',0] = 17
    
    model.Cap_P['loc1','com1_store20_discharge',0].fixed = True
    model.Cap_P['loc1','com1_store20_discharge',0] = 1.777777777777777777
    
    model.Cap_P['loc1','com1_store50',0].fixed = True
    model.Cap_P['loc1','com1_store50',0] = 0
    
    model.Cap_P['loc1','com1_store50_discharge',0].fixed = True
    model.Cap_P['loc1','com1_store50_discharge',0] = 0
    
    model.Cap_P['loc1','procure com1',0].fixed = True
    model.Cap_P['loc1','procure com1',0] = 125
    
    model.Cap_P['loc1','procure optional com1',0].fixed = True
    model.Cap_P['loc1','procure optional com1',0] = 75
    
    model.Cap_P['loc1','sell com1',0].fixed = True
    model.Cap_P['loc1','sell com1',0] = 100
    
    model.X_S['loc1', 'com1_store10_com1_in_stored', 0].fixed = True
    model.X_S['loc1', 'com1_store10_com1_in_stored', 0] = 1
    
    model.X_S['loc1', 'com1_store20_com1_in_stored', 0].fixed = True
    model.X_S['loc1', 'com1_store20_com1_in_stored', 0] = 1
    
    model.X_S['loc1', 'com1_store50_com1_in_stored', 0].fixed = True
    model.X_S['loc1', 'com1_store50_com1_in_stored', 0] = 0
    
    model.Cap_S['loc1', 'com1_store10_com1_in_stored', 0].fixed = True
    model.Cap_S['loc1', 'com1_store10_com1_in_stored', 0] = 40
    
    model.Cap_S['loc1', 'com1_store20_com1_in_stored', 0].fixed = True
    model.Cap_S['loc1', 'com1_store20_com1_in_stored', 0] = 80
    
    model.Cap_S['loc1', 'com1_store50_com1_in_stored', 0].fixed = True
    model.Cap_S['loc1', 'com1_store50_com1_in_stored', 0] = 0.0

In [ ]:
n_avg = 24
cap_factor_avg = pandas.DataFrame(data={'com1_procure': [1]*n_avg + [0]*(_exec_scenarios-n_avg)})
scen_avg, model_avg = build_model(cap_factor=cap_factor_avg)
fix_variables(model=model_avg)
results_avg = solve(scenario=scen_avg, instance=model_avg, solver='gurobi', name='results_avg')

In [ ]:
model_avg.Cap_S.pprint()

In [ ]:
scen_avg.storage_capex_dict

In [ ]:
model_avg.constraint_storage_capex.pprint()

In [ ]:
model_avg.Capex_network.pprint()

In [ ]:
# demand_penalty*sum(value for key, value in results_avg.output['Demand_penalty'].items())
# model_avg.Capex_network[0].value + model_avg.Vopex_network[0].value +  model_avg.Inv_cost_network[0].value

In [ ]:
n_good = 48
cap_factor_good = pandas.DataFrame(data={'com1_procure': [1]*n_good + [0]*(_exec_scenarios-n_good)})
scen_good, model_good = build_model(cap_factor=cap_factor_good)
fix_variables(model=model_good)
results_good = solve(scenario=scen_good, instance=model_good, solver='gurobi', name='results_good')

In [ ]:
model_good.Capex_network.pprint()

In [ ]:
# n_bad = 12
# cap_factor_bad = pandas.DataFrame(data={'com1_procure': [1]*n_bad + [0]*(_exec_scenarios-n_bad)})
# scen_bad, model_bad = build_model(cap_factor=cap_factor_bad)
# fix_variables(model=model_bad)
# results_bad = solve(scenario=scen_bad, instance=model_bad, solver='gurobi', name='results_bad')

In [ ]:
# (results_avg.output['objective'] + results_good.output['objective'] + results_bad.output['objective'])/3

In [ ]:
# model_bad.Demand_penalty_network.pprint()

In [ ]:
# scen_bad.demand_penalty

In [ ]:
# scen_bad.capex_dict

In [ ]:
# model_bad.scales_network.pprint()

In [ ]:
# scale_list(instance=model_bad, scale_levels=scen_bad.network_scale_level+1)

In [ ]:
# scale_tuple(instance=model_bad, scale_levels=len(model_bad.scales))